# Global Sensitivity Analysis

This is a more complex case. Involves varying multiple parameters together and understanding which parameter is the one with the largest influence on results. This translates into a large simulation testing the various possibilities. One could use brute force and test all possible combinations of all parameters, and then run a regression on the results where the dependent variable is the impact of the system and the independent variables are the parameters. However, calculating a result for every possible combination might end up taking too much computation time, especially if the number of tested variables is high. 

## Correlation Analysis
The content below shows how to perform a simple Global Sensitivity Analysis (GSA) for a example product system of a biobased product and using correlation analysis to quantify the defree of sensitivity.

In [1]:
# Import brightway2.5 packages
import bw2calc as bc
import bw2data as bd
import numpy as np
import pandas as pd
from scipy import stats
from lci_to_bw2 import * # import all the functions of this module

In [3]:
bd.projects.set_current('advlca25') # Still working in the same project
bd.databases

Databases dictionary with 9 object(s):
	ALIGNED-biob-prod-dummy
	SAtestdb
	biosphere3
	ecoinvent 3.9.1 conseq
	exldb
	gsa_db
	sa_db
	testbiosphere
	testdb

We start by importing data about a fictional ("dummy") product system for a biobased product.

In [4]:
# Import the dummy product system

# import data from csv

#mydata = pd.read_csv('LCI1-bw-format.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydata = pd.read_csv('ALIGNED-LCI-biobased-product-dummy.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydata.head()

# keep only the columns not needed
mydb = mydata[['Activity database','Activity code','Activity name','Activity unit','Activity type',
               'Exchange database','Exchange input','Exchange amount','Exchange unit','Exchange type',
               'Exchange uncertainty type','Exchange loc','Exchange scale','Exchange negative', 'Exchange minimum', 'Exchange maximum', 
               'Simapro name',	'Simapro unit', 'Simapro type']].copy()

mydb['Exchange uncertainty type'] = mydb['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
# Note: to avoid having both nan and values in the uncertainty column I use zero as default

#print(mydb.head())

In [5]:
# Create dictionary in bw format and write database to disk. 
# Shut down all other notebooks using the same project before doing this
bw2_db = lci_to_bw2(mydb) # a function from the lci_to_bw2 module

# write database
bd.Database('ALIGNED-biob-prod-dummy').write(bw2_db)

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1752.01it/s]

Vacuuming database 


The product system includes different activities such as the production, use, and end of life of the biobased product.

In [6]:
# check what foreground activities are included
for act in bd.Database('ALIGNED-biob-prod-dummy'):
    print(act, act['code'])

'Biomass-processing' (kilogram, None, None) 403a5c32-c769-46fc-8b9a-74b8eb3c79d1
'Biobased-product-manufacturing' (kilogram, None, None) a37d149a-6508-4563-8af6-e5a39b4176df
'Biomass-growth' (kilogram, None, None) a7d34649-9c10-4423-bac3-ecab9b43b20c
'Biobased-product-use' (year, None, None) f9eabf64-b899-40c0-9f9f-2009dbb0a0b2
'Biobased-product-eol' (kilogram, None, None) c8301e73-d521-4a89-998b-30b7e7751011


In [7]:
# More info 
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
myact._data

{'name': 'Biobased-product-use',
 'unit': 'year',
 'type': 'process',
 'database': 'ALIGNED-biob-prod-dummy',
 'code': 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2',
 'id': 23611}

In [8]:
# Uncertainty is also there
list(myact.exchanges())[1]._data

{'input': ('ALIGNED-biob-prod-dummy', 'a37d149a-6508-4563-8af6-e5a39b4176df'),
 'amount': 50.0,
 'unit': 'kilogram',
 'type': 'technosphere',
 'uncertainty type': 4,
 'minimum': 37.5,
 'maximum': 62.5,
 'Simapro name': 'Biobased-product-manufacturing',
 'Simapro unit': 'kg',
 'output': ('ALIGNED-biob-prod-dummy', 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2')}

We calculate a static climate impact score for the fictional biobased product, to be used for reference later on.

In [9]:
# calculation of static LCA score
mymethod = ('IPCC 2013', 'climate change', 'global warming potential (GWP100)')
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
functional_unit = {myact: 1}
LCA = bc.LCA(functional_unit, mymethod)
LCA.lci()
LCA.lcia()
print(LCA.score)

121.67148771458245


### Now perform global sensitivity analysis

The procedure is in three steps.

1) A set of model input parameters is chosen. These are **values of specific exchanges**. A sample of values is produced for each model input, in this case, only 5 values.
2) A simulation is performed. Initial prameter values are substituted with those in the sample, iteratively, and new model outputs, that are LCA scores, are calculated.
3) A correlation is estimated between model input values and output values.

#### Step 1
A sample of values for each parameter.

In [10]:
# Can be done in many ways, here very basic using lists.
par1_values = [-1.25, -1.10, -1, -0.9, -0.75] # In bomass growth, value of CO2 uptake
par2_values = [0.1, 0.3, 0.5, 0.7, 0.9] # In biomass processing, amount of energy used.
par3_values = [20,30,40,50,60]  # In use of product, amount of manufactured product needed
par4_values = [1.25, 1.10, 1, 0.9, 0.75] # In use of product, amount of manufactured product needed

Associate the values to specific exchanges using the coordinates (column and row) of the technosphere (A) and biosphere (B) matrices

In [11]:
param_samples = [(('ALIGNED-biob-prod-dummy', 'a7d34649-9c10-4423-bac3-ecab9b43b20c'), 
  ('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'), par1_values), # In bomass growth, value of CO2 uptake
(('ALIGNED-biob-prod-dummy', '403a5c32-c769-46fc-8b9a-74b8eb3c79d1'),
 ('ecoinvent 3.9 conseq', 'f4dc7d2b1d70e6c0f929ec5231c085e0'), par2_values), # In biomass processing, amount of energy used.
 (('ALIGNED-biob-prod-dummy', 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2'),
  ('ALIGNED-biob-prod-dummy', 'a37d149a-6508-4563-8af6-e5a39b4176df'), par3_values), # In use of product, amount of manufactured product needed
  (('ALIGNED-biob-prod-dummy', 'c8301e73-d521-4a89-998b-30b7e7751011'),
   ('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'), par4_values)] # In end of life, amount of CO2 released

#### (technical note)

We can do the same in a more elegant way, taking the data directly from the BW database that we just created...

In [12]:
# In biomass growth, value of CO2 uptake
par1 = list(bd.Database('ALIGNED-biob-prod-dummy').get('a7d34649-9c10-4423-bac3-ecab9b43b20c').exchanges())[3]
# In biomass processing, amount of energy used.
par2 = list(bd.Database('ALIGNED-biob-prod-dummy').get('403a5c32-c769-46fc-8b9a-74b8eb3c79d1').exchanges())[1]
# In use of product, amount of manufactured product needed
par3 = list(bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2').exchanges())[1]
# In use of product, amount of manufactured product needed
par4 = list(bd.Database('ALIGNED-biob-prod-dummy').get('c8301e73-d521-4a89-998b-30b7e7751011').exchanges())[2]

n_iter = 5
param_samples = [(par1['output'],par1['input'], par1.random_sample(n = n_iter)),
                 (par2['output'],par2['input'], par2.random_sample(n = n_iter)),
                 (par3['output'],par3['input'], par3.random_sample(n = n_iter)),
                 (par4['output'],par4['input'], par4.random_sample(n = n_iter))]

Let's look at the samples obtained

In [13]:
param_samples

[(('ALIGNED-biob-prod-dummy', 'a7d34649-9c10-4423-bac3-ecab9b43b20c'),
  ('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'),
  array([-0.94008983, -0.8699042 , -1.24487299, -0.97397597, -1.1907887 ])),
 (('ALIGNED-biob-prod-dummy', '403a5c32-c769-46fc-8b9a-74b8eb3c79d1'),
  ('ecoinvent 3.9.1 conseq', 'f4dc7d2b1d70e6c0f929ec5231c085e0'),
  array([0.48602327, 0.52151973, 0.49501432, 0.606012  , 0.45231842])),
 (('ALIGNED-biob-prod-dummy', 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2'),
  ('ALIGNED-biob-prod-dummy', 'a37d149a-6508-4563-8af6-e5a39b4176df'),
  array([44.96312104, 62.09817233, 44.62648947, 43.75094756, 53.85788237])),
 (('ALIGNED-biob-prod-dummy', 'c8301e73-d521-4a89-998b-30b7e7751011'),
  ('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'),
  array([1.23169336, 1.11548435, 0.78135885, 1.02840774, 1.02881701]))]

#### Step 2

Iteration through all parameter samples. Input values are replaced and new impact scores are calculated at each iteration.

In [14]:
# testing the loop that will be used for the simulation, iterate through parameter values
    
for i in range(0,n_iter): # iterate 5 times...
    for s in param_samples: # and for each paramater...
        
        # Get the 'id' using the activity object
        col_id = bd.Database(s[0][0]).get(s[0][1]).id
        row_id = bd.Database(s[1][0]).get(s[1][1]).id
        
        # Use the id and the mapping dictionaries to find matrix row and columns
        if s[1][0] == "biosphere3":
            col = LCA.activity_dict[col_id] # find column index of A matrix for the activity
            row = LCA.biosphere_dict[row_id] # find row index of B matrix for the exchange
            print(LCA.biosphere_matrix[row,col], s[2][i]) # print the initial and final value (to be substituted) ????
        
        else:
            col = LCA.activity_dict[col_id] # find column index of A matrix for the activity
            row = LCA.activity_dict[row_id] # find row index of A matrix for the exchange
            print(LCA.technosphere_matrix[row,col], -s[2][i]) # CHANGE OF SIGN!
        
    print("* * *")

-1.0 -0.9400898298528071
-0.5 -0.48602326965975096
-50.0 -44.96312104319368
1.0 1.2316933648038761
* * *
-1.0 -0.8699042041934981
-0.5 -0.5215197291697707
-50.0 -62.098172327364686
1.0 1.1154843489158166
* * *
-1.0 -1.2448729915176875
-0.5 -0.49501432255318534
-50.0 -44.626489467940324
1.0 0.7813588464915562
* * *
-1.0 -0.9739759715538552
-0.5 -0.6060119983235702
-50.0 -43.75094755949158
1.0 1.0284077385584582
* * *
-1.0 -1.1907886987662732
-0.5 -0.4523184220355385
-50.0 -53.85788237085855
1.0 1.028817007330332
* * *


In [15]:
# Implementing the loop

GSA_value_results = []

for i in range(0,n_iter): 
    for s in param_samples:
        col_id = bd.Database(s[0][0]).get(s[0][1]).id
        row_id = bd.Database(s[1][0]).get(s[1][1]).id
        
        if s[1][0] == "biosphere3":
            col = LCA.activity_dict[col_id] # find column index of A matrix for the activity
            row = LCA.biosphere_dict[row_id] # find row index of B matrix for the exchange
            LCA.biosphere_matrix[row,col] = s[2][i] # substitute the value
        else:
            col = LCA.activity_dict[col_id] # find column index of A matrix for the activity
            row = LCA.activity_dict[row_id] # find row index of A matrix for the exchange
            LCA.technosphere_matrix[row,col] = -s[2][i] # substitute the value
                
    LCA.redo_lci() # uses the new A matrix
    LCA.lcia()
    print(LCA.score)
    GSA_value_results.append(LCA.score)

127.47433450820517
148.60406051512743
97.67956238263707
114.92783075977592
123.37221292412063


#### Step 3

Now we have both the input and output values and we can look at their correlation.

To estimate the degree of correlation use a Pearson coefficient of linear relationship between two sets of values.

See here for info: 
https://en.wikipedia.org/wiki/Pearson_correlation_coefficient
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html

This approach has for example been used in:

_Kim, A., Mutel, C., Froemelt, A., 2021. Robust high-dimensional screening. Environmental Modelling & Software 105270._ https://doi.org/10.1016/j.envsoft.2021.105270


In [16]:
# calcualte correlation

corr_data = pd.DataFrame([i[2] for i in param_samples], index = ['par1','par2', 'par3', 'par4']).T
corr_data['GWI'] = GSA_value_results
corr_data.corr()

,par1,par2,par3,par4,GWI
par1,1.000000,0.464900,0.316500,0.789531,0.769329
par2,0.464900,1.000000,-0.259602,-0.007492,-0.051742
par3,0.316500,-0.259602,1.000000,0.258492,0.801096
par4,0.789531,-0.007492,0.258492,1.000000,0.766056
GWI,0.769329,-0.051742,0.801096,0.766056,1.000000


In this specific case "par3"  is the parameter to which the results are most sensitive to.

In [17]:
# what was "par3" again?
par3 # amount of manufactured product need din the use stage

Exchange: 50.0 kilogram 'Biobased-product-manufacturing' (kilogram, None, None) to 'Biobased-product-use' (year, None, None)>

## FAST

The content below shows how to perform a simple Global Sensitivity Analysis (GSA) applying the FAST, a variance-based sensitivity analysis method, for an example product system of a biobased product.

In [18]:
# importing packages
import bw2calc as bc
import bw2data as bd
import pandas as pd
import numpy as np
from scipy import stats
from lci_to_bw2 import * # import all the functions of this module

In [20]:
# open a project with ecoinvent v.3.9.1 consequential system model
bd.projects.set_current('advlca25')

We start by importing data about a fictional ("dummy") product system for a biobased product.

In [21]:
# Import the dummy product system

# import data from csv
mydata = pd.read_csv('ALIGNED-LCI-biobased-product-dummy.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydata.head()

# keep only the columns not needed
mydb = mydata[['Activity database','Activity code','Activity name','Activity unit','Activity type',
               'Exchange database','Exchange input','Exchange amount','Exchange unit','Exchange type',
               'Exchange uncertainty type','Exchange loc','Exchange scale','Exchange negative', 'Exchange minimum', 'Exchange maximum', 
               'Simapro name',	'Simapro unit', 'Simapro type']].copy()

mydb['Exchange uncertainty type'] = mydb['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
# Note: to avoid having both nan and values in the uncertainty column I use zero as default

#print(mydb.head())

In [22]:
# Create dictionary in bw format and write database to disk. 
# Shut down all other notebooks using the same project before doing this
bw2_db = lci_to_bw2(mydb) # a function from the lci_to_bw2 module

# write database
bd.Database('ALIGNED-biob-prod-dummy').write(bw2_db)

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 3004.95it/s]

Vacuuming database 


The product system includes different activities such as the production, use, and end of life of the biobased product.

In [23]:
# check what foreground activities are included
for act in bd.Database('ALIGNED-biob-prod-dummy'):
    print(act, act['code'])

'Biobased-product-use' (year, None, None) f9eabf64-b899-40c0-9f9f-2009dbb0a0b2
'Biomass-growth' (kilogram, None, None) a7d34649-9c10-4423-bac3-ecab9b43b20c
'Biomass-processing' (kilogram, None, None) 403a5c32-c769-46fc-8b9a-74b8eb3c79d1
'Biobased-product-manufacturing' (kilogram, None, None) a37d149a-6508-4563-8af6-e5a39b4176df
'Biobased-product-eol' (kilogram, None, None) c8301e73-d521-4a89-998b-30b7e7751011


In [24]:
# More info 
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
myact._data

{'name': 'Biobased-product-use',
 'unit': 'year',
 'type': 'process',
 'database': 'ALIGNED-biob-prod-dummy',
 'code': 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2',
 'id': 23611}

In [25]:
# Uncertainty is also there
list(myact.exchanges())[1]._data

{'input': ('ALIGNED-biob-prod-dummy', 'a37d149a-6508-4563-8af6-e5a39b4176df'),
 'amount': 50.0,
 'unit': 'kilogram',
 'type': 'technosphere',
 'uncertainty type': 4,
 'minimum': 37.5,
 'maximum': 62.5,
 'Simapro name': 'Biobased-product-manufacturing',
 'Simapro unit': 'kg',
 'output': ('ALIGNED-biob-prod-dummy', 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2')}

We calculate a static climate impact score for the fictional biobased product, to be used for reference later on.

In [26]:
# calculation of static LCA score
mymethod = ('IPCC 2013', 'climate change', 'global warming potential (GWP100)')
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
functional_unit = {myact: 1}
LCA = bc.LCA(functional_unit, mymethod)
LCA.lci()
LCA.lcia()
print(LCA.score)

121.67148771458245


### Now perform sensitivity analysis

The procedure is in three steps.

1) A set of model input parameters is chosen. These are **values of specific exchanges**. A sample of values is produced for each model input **using a specific and efficient sampling design** (the **"problem"** below)
2) A simulation is performed. Initial prameter values are substituted with those in the sample, iteratively, and new model outputs, that are LCA scores, are calculated.
3) **A sensitivity index is calculated** using model input values (the "problem") and output values.

#### Step 1
Obtain a sample of values for each parameter.

This is a  pseudo-random sample obtained using a sampling method from the SALib package on the problem previously defined.

it is defined as a __FAST__ sample ("Fourier Amplitude Sensitivity Analysis"), another alternative to Sobol method.

Sobol and FAST sampling methods use different pseudo-random patterns to cover the space of parameters, and different calculations for the sensitivity indices.

Besides these differences, the (extended) FAST method is, just like the Sobol one, a variance-based method for sensitivity analysis. It provides the same information as the Sobol method but with higher computational efficiency.

For further information and comparison see: 

_Saltelli, A.; Tarantola, S.; Chan, K. P. S. A Quantitative Model-Independent Method for Global Sensitivity Analysis of Model Output. Technometrics 1999, 41 (1), 39–56._ https://doi.org/10.1080/00401706.1999.10485594.


In [27]:
# need to import the SALib library and relaive methods for sensitvity analysis
# do 'pip install SALib' in the brightway environment

from SALib.sample import saltelli
from SALib.sample import fast_sampler
from SALib.analyze import sobol
from SALib.analyze import fast


Define the "problem" for the GSA analysis as indicated in the SALib library.
Uniform distributions for the uncertainty of for each input parameter are here assumed.

In [28]:
problem = { 'num_vars': 4, # number of variables
            'names': ['par1', 'par2', 'par3', 'par4'], # names of variables, same as parameters
            'bounds': [[-1.25, -0.75], # careful here to what is the lower bound with negative values...
                       [0.1, 0.9],  
                       [20, 60],
                       [0.75, 1.25]] ,
           'dists':["unif","unif","unif","unif"] } # all uniform distributions

In [29]:
# FAST sampler creates N*P parameters sets with N=sample size, P = number of parameters
param_values_FAST = fast_sampler.sample(problem, 200)
print(param_values_FAST.shape)

param_values_FAST

(800, 4)


array([[-0.83186667,  0.76901333, 53.45066667,  1.16813333],
       [-0.78813333,  0.77701333, 54.25066667,  1.18313333],
       [-0.90813333,  0.78501333, 55.05066667,  1.19813333],
       ...,
       [-1.13018109,  0.31571026, 31.98551281,  1.21481891],
       [-1.13518109,  0.29971026, 30.78551281,  1.09481891],
       [-1.14018109,  0.28371026, 29.58551281,  0.97481891]])

Associate the values to specific exchanges using the coordinates (column and row) of the technosphere (A) and biosphere (B) matrices, taking the data directly from the BW database that we just created.

In [30]:
# In biomass growth, value of CO2 uptake
par1 = list(bd.Database('ALIGNED-biob-prod-dummy').get('a7d34649-9c10-4423-bac3-ecab9b43b20c').exchanges())[3]
# In biomass processing, amount of energy used.
par2 = list(bd.Database('ALIGNED-biob-prod-dummy').get('403a5c32-c769-46fc-8b9a-74b8eb3c79d1').exchanges())[1]
# In use of product, amount of manufactured product needed
par3 = list(bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2').exchanges())[1]
# In use of product, amount of manufactured product needed
par4 = list(bd.Database('ALIGNED-biob-prod-dummy').get('c8301e73-d521-4a89-998b-30b7e7751011').exchanges())[2]

n_iter = len(param_values_FAST)
param_samples = [(par1['output'],par1['input'], [i[0] for i in param_values_FAST]),
                 (par2['output'],par2['input'], [i[1] for i in param_values_FAST]),
                 (par3['output'],par3['input'], [i[2] for i in param_values_FAST]),
                 (par4['output'],par4['input'], [i[3] for i in param_values_FAST])]

#### Step 2

Iteration through all parameter samples. New LCA scores are calculated for each combination of values.

With 800 iterations, **this will take some time**

In [31]:
# Implementing the loop

GSA_value_results = []

for i in range(0,len(param_values_FAST)): 
    for s in param_samples:
        # Get the 'id' using the activity object
        col_id = bd.Database(s[0][0]).get(s[0][1]).id
        row_id = bd.Database(s[1][0]).get(s[1][1]).id
        
        # Use the id and the mapping dictionaries to find matrix row and columns
        if s[1][0] == "biosphere3":
            col = LCA.activity_dict[col_id] # find column index of A matrix for the activity
            row = LCA.biosphere_dict[row_id] # find row index of B matrix for the exchange
            LCA.biosphere_matrix[row,col] = s[2][i] # substitute the value
        else:
            col = LCA.activity_dict[col_id] # find column index of A matrix for the activity
            row = LCA.activity_dict[row_id] # find row index of A matrix for the exchange
            LCA.technosphere_matrix[row,col] = -s[2][i] # substitute the value
        
    LCA.redo_lci() # uses the new A matrix
    LCA.lcia()
    #print(LCA.score)
    GSA_value_results.append(LCA.score)

#### Step 3

Now we have both the input and output values and we can feed the problem and the results to the Sobol function to obtain the FAST indices.

First we look at the parameters and the result



In [32]:
# Organize data first to give a look at what we have done: a sample of input values and a corresponding output value
fast_data = pd.DataFrame([i[2] for i in param_samples], index = ['par0','par1', 'par2', 'par3']).T
fast_data['GWI'] = GSA_value_results
fast_data.head(10) # show only the first ten samples

,par0,par1,par2,par3,GWI
0,-0.831867,0.769013,53.450667,1.168133,139.740957
1,-0.788133,0.777013,54.250667,1.183133,142.887983
2,-0.908133,0.785013,55.050667,1.198133,141.563444
3,-1.028133,0.793013,55.850667,1.213133,140.143166
4,-1.148133,0.801013,56.650667,1.228133,138.627151
5,-1.231867,0.809013,57.450667,1.243133,138.057169
6,-1.111867,0.817013,58.250667,1.241867,142.540932
7,-0.991867,0.825013,59.050667,1.226867,146.434289
8,-0.871867,0.833013,59.850667,1.211867,150.423909
9,-0.751867,0.841013,59.349333,1.196867,152.500799


Now we calculate the SI index

In [33]:
si = fast.analyze(problem, np.array(GSA_value_results), print_to_console=True) # must use np.array

            S1        ST   S1_conf   ST_conf
par1  0.025341  0.029323  0.063048  0.091228
par2  0.000112  0.001037  0.075100  0.083699
par3  0.821179  0.825417  0.072916  0.091960
par4  0.157746  0.160019  0.073431  0.084295


In this specific case "par3"  is the parameter to which the results are most sensitive to, this because it has the highest value for "S1" which is the first order effect.

This is determined with relativelygood confidence as the error (value of "S1_conf") is small compared to the coefficient.

Note also that "par3" is also the parameter withi highest sensitivity in the total model, i.e. when in combinatino with other parameters, as observed by the highest value of "ST".

In [34]:
# In this specific case "par3" is the parameter to which the results ar emost sensitive to, and values have high confidente (small error)
# what was "par3" again?
par3 # amount of manufactured product need din the use stage

Exchange: 50.0 kilogram 'Biobased-product-manufacturing' (kilogram, None, None) to 'Biobased-product-use' (year, None, None)>